In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Cargar dataset de vinos
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

# Descripción de las variables
print("=== DESCRIPCIÓN DE VARIABLES DEL DATASET DE VINOS ===")
print("\nDescripción general del dataset:")
print(f"Número de muestras: {X.shape[0]}")
print(f"Número de características: {X.shape[1]}")
print(f"Clases: {np.unique(y)}")
print(f"Nombres de clases: {wine.target_names}")
print(f"Distribución de clases: {pd.Series(y).value_counts().to_dict()}")

# Crear un DataFrame con la descripción de las variables
variable_descriptions = {
    'alcohol': 'Contenido de alcohol (% vol)',
    'malic_acid': 'Ácido málico (g/L)',
    'ash': 'Ceniza (g/L)',
    'alcalinity_of_ash': 'Alcalinidad de la ceniza (pH)',
    'magnesium': 'Magnesio (mg/L)',
    'total_phenols': 'Fenoles totales (mg/L)',
    'flavanoids': 'Flavonoides (mg/L)',
    'nonflavanoid_phenols': 'Fenoles no flavonoides (mg/L)',
    'proanthocyanins': 'Proantocianinas (mg/L)',
    'color_intensity': 'Intensidad de color (absorbancia)',
    'hue': 'Tonalidad (absorbancia)',
    'od280/od315_of_diluted_wines': 'OD280/OD315 de vinos diluidos (índice)',
    'proline': 'Prolina (mg/L)'
}

# Estadísticas descriptivas
stats = X.describe().T
stats['descripcion'] = pd.Series(variable_descriptions)
stats = stats[['descripcion', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']]
stats = stats.rename(columns={
    'mean': 'Media',
    'std': 'Desv. Estándar',
    'min': 'Mínimo',
    '25%': 'Cuartil 1',
    '50%': 'Mediana',
    '75%': 'Cuartil 3',
    'max': 'Máximo',
    'descripcion': 'Descripción'
})

print("\n=== ESTADÍSTICAS DESCRIPTIVAS DE LAS VARIABLES ===")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
print(stats.round(2))

# Visualizar la distribución de cada variable por clase
plt.figure(figsize=(15, 10))
for i, feature in enumerate(X.columns):
    plt.subplot(4, 4, i+1)
    for target in np.unique(y):
        plt.hist(X[feature][y == target], alpha=0.5, label=wine.target_names[target], bins=15)
    plt.xlabel(feature)
    plt.legend(loc='upper right')
    plt.tight_layout()
plt.savefig('distribucion_variables_por_clase.png')
plt.close()

# Correlación entre variables
plt.figure(figsize=(14, 10))
correlation = X.corr(method='spearman')
mask = np.triu(correlation)
sns.heatmap(correlation, annot=True, fmt=".2f", cmap='coolwarm', mask=mask)
plt.title('Matriz de Correlación entre Variables')
plt.tight_layout()
plt.savefig('matriz_correlacion_detallada.png')
plt.close()

# Importancia de las variables para la clasificación
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif

# Importancia basada en Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X, y)
importances = pd.DataFrame({
    'Variable': X.columns,
    'Importancia RF': rf.feature_importances_
}).sort_values('Importancia RF', ascending=False)

# Importancia basada en Información Mutua
mi_scores = mutual_info_classif(X, y)
mi_importances = pd.DataFrame({
    'Variable': X.columns,
    'Información Mutua': mi_scores
}).sort_values('Información Mutua', ascending=False)

# Combinar ambas métricas
importances = importances.merge(mi_importances, on='Variable')
importances['Importancia Promedio'] = (importances['Importancia RF'] +
                                      importances['Información Mutua'] / importances['Información Mutua'].max()) / 2
importances = importances.sort_values('Importancia Promedio', ascending=False)

print("\n=== IMPORTANCIA DE LAS VARIABLES PARA LA CLASIFICACIÓN ===")
print(importances.round(4))

# Visualizar importancia de variables
plt.figure(figsize=(12, 6))
plt.barh(importances['Variable'], importances['Importancia Promedio'], color='teal')
plt.xlabel('Importancia Promedio')
plt.ylabel('Variable')
plt.title('Importancia de Variables para la Clasificación')
plt.tight_layout()
plt.savefig('importancia_variables.png')
plt.close()

# Visualizar distribución de clases
plt.figure(figsize=(8, 6))
class_counts = pd.Series(y).value_counts().sort_index()
plt.bar(wine.target_names, class_counts, color=['red', 'green', 'blue'])
plt.xlabel('Clase de Vino')
plt.ylabel('Número de Muestras')
plt.title('Distribución de Clases en el Dataset')
for i, count in enumerate(class_counts):
    plt.text(i, count + 1, str(count), ha='center')
plt.tight_layout()
plt.savefig('distribucion_clases_detallada.png')
plt.close()

# Análisis de separabilidad de clases con PCA
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(10, 8))
for target in np.unique(y):
    plt.scatter(X_pca[y == target, 0], X_pca[y == target, 1],
                label=wine.target_names[target], alpha=0.7,
                edgecolors='k', s=80)
plt.xlabel(f'Componente Principal 1 ({pca.explained_variance_ratio_[0]:.2%} varianza)')
plt.ylabel(f'Componente Principal 2 ({pca.explained_variance_ratio_[1]:.2%} varianza)')
plt.title('Separación de Clases de Vino usando PCA')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('separacion_clases_pca.png')
plt.close()

# Crear un documento con toda la información
report = pd.DataFrame({
    'Variable': X.columns,
    'Descripción': [variable_descriptions[col] for col in X.columns],
    'Media': X.mean().values,
    'Desv. Estándar': X.std().values,
    'Importancia': importances.set_index('Variable')['Importancia Promedio'].reindex(X.columns).values
})

# Guardar el informe
report.to_csv('descripcion_variables_vino.csv', index=False)
report.to_excel('descripcion_variables_vino.xlsx', index=False)

print("\nInforme de descripción de variables guardado en 'descripcion_variables_vino.csv' y 'descripcion_variables_vino.xlsx'")

=== DESCRIPCIÓN DE VARIABLES DEL DATASET DE VINOS ===

Descripción general del dataset:
Número de muestras: 178
Número de características: 13
Clases: [0 1 2]
Nombres de clases: ['class_0' 'class_1' 'class_2']
Distribución de clases: {1: 71, 0: 59, 2: 48}

=== ESTADÍSTICAS DESCRIPTIVAS DE LAS VARIABLES ===
                                                         Descripción   Media  Desv. Estándar  Mínimo  Cuartil 1  Mediana  Cuartil 3   Máximo
alcohol                                 Contenido de alcohol (% vol)   13.00            0.81   11.03      12.36    13.05      13.68    14.83
malic_acid                                        Ácido málico (g/L)    2.34            1.12    0.74       1.60     1.87       3.08     5.80
ash                                                     Ceniza (g/L)    2.37            0.27    1.36       2.21     2.36       2.56     3.23
alcalinity_of_ash                      Alcalinidad de la ceniza (pH)   19.49            3.34   10.60      17.20    19.50      21.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_wine

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold

from sklearn.tree import DecisionTreeClassifier, plot_tree

from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer

from sklearn.preprocessing import StandardScaler, PowerTransformer, QuantileTransformer, RobustScaler

from sklearn.feature_selection import mutual_info_classif

from imblearn.pipeline import Pipeline as ImbPipeline

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTEENN, SMOTETomek

import warnings
warnings.filterwarnings('ignore')

# Cargar dataset de vinos (clasificación multiclase)
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Verificar el desbalance inicial
print("=== DISTRIBUCIÓN ORIGINAL DE CLASES (TRAIN) ===")
train_class_counts = pd.Series(y_train).value_counts()
print(train_class_counts)
print(f"Proporción: {pd.Series(y_train).value_counts(normalize=True).round(2)}")

# Función para evaluar y mostrar resultados
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)

    # F1-Score
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_by_class = f1_score(y_test, y_pred, average=None)

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)

    # Reporte de clasificación
    report = classification_report(y_test, y_pred)

    # Visualizar matriz de confusión
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=np.unique(y),
                yticklabels=np.unique(y))
    plt.title(f'Matriz de Confusión - {model_name}')
    plt.xlabel('Predicción')
    plt.ylabel('Valor Real')
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_{model_name.replace(" ", "_").lower()}.png')
    plt.close()

    # Resultados
    print(f"\n=== RESULTADOS DEL MODELO: {model_name} ===")
    print(f"F1-Score (weighted): {f1:.4f}")
    print(f"F1-Score por clase: {f1_by_class}")
    print("\nMatriz de Confusión:")
    print(cm)
    print("\nReporte de Clasificación:")
    print(report)

    return f1, f1_by_class, cm

# 1. MODELO ORIGINAL (BASELINE)
print("\n=== MODELO ORIGINAL (BASELINE) ===")
dt_original = DecisionTreeClassifier(random_state=42)
dt_original.fit(X_train, y_train)
f1_original, f1_by_class_original, _ = evaluate_model(dt_original, X_test, y_test, "Original")

# 2. ESTRATEGIA 1: TRANSFORMACIÓN AVANZADA DE CARACTERÍSTICAS
print("\n=== ESTRATEGIA 1: TRANSFORMACIÓN AVANZADA DE CARACTERÍSTICAS ===")

# Probar diferentes transformadores para encontrar el mejor
transformers = {
    'PowerTransformer': PowerTransformer(method='yeo-johnson'),
    'QuantileTransformer': QuantileTransformer(output_distribution='normal'),
    'RobustScaler': RobustScaler()
}

best_transformer = None
best_f1 = 0

for name, transformer in transformers.items():
    # Transformar datos
    X_train_transformed = transformer.fit_transform(X_train)
    X_test_transformed = transformer.transform(X_test)

    # Entrenar modelo básico
    dt = DecisionTreeClassifier(random_state=42)
    dt.fit(X_train_transformed, y_train)

    # Evaluar
    y_pred = dt.predict(X_test_transformed)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"F1-Score con {name}: {f1:.4f}")

    if f1 > best_f1:
        best_f1 = f1
        best_transformer = transformer

print(f"\nMejor transformador: {type(best_transformer).__name__}")

# Aplicar el mejor transformador
X_train_transformed = best_transformer.fit_transform(X_train)
X_test_transformed = best_transformer.transform(X_test)

# 3. ESTRATEGIA 2: OPTIMIZACIÓN EXHAUSTIVA DE HIPERPARÁMETROS
print("\n=== ESTRATEGIA 2: OPTIMIZACIÓN EXHAUSTIVA DE HIPERPARÁMETROS ===")

# Crear un scorer personalizado basado en F1
f1_scorer = make_scorer(f1_score, average='weighted')

# Grid de hiperparámetros más exhaustivo
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 15, 20, 25, 30],
    'min_samples_split': [2, 3, 5, 7, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'max_features': [None, 'sqrt', 'log2', 0.7, 0.8, 0.9],
    'class_weight': [None, 'balanced'],
    'ccp_alpha': [0.0, 0.01, 0.02, 0.03, 0.05]  # Pruning para evitar overfitting
}

# Usar RandomizedSearchCV para explorar más eficientemente
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_distributions=param_grid,
    n_iter=100,  # Número de combinaciones a probar
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring=f1_scorer,
    n_jobs=-1,
    random_state=42
)

random_search.fit(X_train_transformed, y_train)
print(f"\nMejores parámetros: {random_search.best_params_}")
print(f"Mejor F1-Score CV: {random_search.best_score_:.4f}")

# Evaluar modelo con hiperparámetros optimizados
dt_optimized = random_search.best_estimator_
f1_optimized, f1_by_class_optimized, _ = evaluate_model(dt_optimized, X_test_transformed, y_test, "Hiperparámetros Optimizados")

# 4. ESTRATEGIA 3: BALANCEO AVANZADO + ÁRBOL DE DECISIÓN OPTIMIZADO
print("\n=== ESTRATEGIA 3: BALANCEO AVANZADO + ÁRBOL DE DECISIÓN OPTIMIZADO ===")

# Probar diferentes técnicas de balanceo
balancers = {
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTEENN': SMOTEENN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42)
}

best_balancer = None
best_f1_balancer = 0

for name, balancer in balancers.items():
    # Crear pipeline con balanceo y árbol optimizado
    pipeline = ImbPipeline([
        ('transformer', best_transformer),
        ('balancer', balancer),
        ('classifier', DecisionTreeClassifier(**random_search.best_params_, random_state=42))
    ])

    # Entrenar
    pipeline.fit(X_train, y_train)

    # Evaluar
    y_pred = pipeline.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')

    print(f"F1-Score con {name}: {f1:.4f}")

    if f1 > best_f1_balancer:
        best_f1_balancer = f1
        best_balancer = balancer

print(f"\nMejor técnica de balanceo: {type(best_balancer).__name__}")

# Crear pipeline final con el mejor balanceador
pipeline_final = ImbPipeline([
    ('transformer', best_transformer),
    ('balancer', best_balancer),
    ('classifier', DecisionTreeClassifier(**random_search.best_params_, random_state=42))
])

# Entrenar pipeline final
pipeline_final.fit(X_train, y_train)

# Evaluar pipeline final
f1_pipeline, f1_by_class_pipeline, _ = evaluate_model(pipeline_final, X_test, y_test, "Pipeline Final")

# 5. ESTRATEGIA 4: FEATURE ENGINEERING MANUAL + ÁRBOL OPTIMIZADO
print("\n=== ESTRATEGIA 4: FEATURE ENGINEERING MANUAL + ÁRBOL OPTIMIZADO ===")

# Crear características adicionales basadas en conocimiento del dominio
X_train_fe = X_train.copy()
X_test_fe = X_test.copy()

# Ratios entre características (pueden capturar relaciones importantes)
X_train_fe['alcohol_ash_ratio'] = X_train['alcohol'] / X_train['ash']
X_test_fe['alcohol_ash_ratio'] = X_test['alcohol'] / X_test['ash']

X_train_fe['flavanoids_phenols_ratio'] = X_train['flavanoids'] / X_train['total_phenols']
X_test_fe['flavanoids_phenols_ratio'] = X_test['flavanoids'] / X_test['total_phenols']

X_train_fe['color_hue_ratio'] = X_train['color_intensity'] / X_train['hue']
X_test_fe['color_hue_ratio'] = X_test['color_intensity'] / X_test['hue']

# Productos entre características importantes (según información mutua)
mi_scores = mutual_info_classif(X_train, y_train)
mi_features = pd.DataFrame({'Feature': X.columns, 'MI_Score': mi_scores})
top_features = mi_features.sort_values('MI_Score', ascending=False)['Feature'].iloc[:3].values

for i, feat1 in enumerate(top_features):
    for feat2 in top_features[i+1:]:
        feat_name = f"{feat1}_{feat2}_product"
        X_train_fe[feat_name] = X_train[feat1] * X_train[feat2]
        X_test_fe[feat_name] = X_test[feat1] * X_test[feat2]

# Transformar datos con feature engineering
X_train_fe_transformed = best_transformer.fit_transform(X_train_fe)
X_test_fe_transformed = best_transformer.transform(X_test_fe)

# Entrenar modelo con feature engineering
dt_fe = DecisionTreeClassifier(**random_search.best_params_, random_state=42)
dt_fe.fit(X_train_fe_transformed, y_train)

# Evaluar modelo con feature engineering
f1_fe, f1_by_class_fe, _ = evaluate_model(dt_fe, X_test_fe_transformed, y_test, "Feature Engineering")

# 6. ESTRATEGIA 5: COMBINACIÓN DE TODAS LAS TÉCNICAS
print("\n=== ESTRATEGIA 5: COMBINACIÓN DE TODAS LAS TÉCNICAS ===")

# Crear pipeline final con todas las mejoras
pipeline_combined = ImbPipeline([
    ('transformer', best_transformer),
    ('balancer', best_balancer),
    ('classifier', DecisionTreeClassifier(**random_search.best_params_, random_state=42))
])

# Entrenar con datos que incluyen feature engineering
pipeline_combined.fit(X_train_fe, y_train)

# Evaluar pipeline combinado
f1_combined, f1_by_class_combined, _ = evaluate_model(pipeline_combined, X_test_fe, y_test, "Combinación Final")

# Visualizar el árbol final para entender su estructura
plt.figure(figsize=(20, 10))
tree_model = pipeline_combined.named_steps['classifier']
plot_tree(tree_model,
          feature_names=X_test_fe.columns,
          class_names=[str(i) for i in range(3)],
          filled=True,
          rounded=True,
          max_depth=3)  # Limitar profundidad para visualización
plt.savefig('arbol_decision_final.png', dpi=300, bbox_inches='tight')
plt.close()

# Comparar todos los modelos
print("\n=== COMPARACIÓN DE TODOS LOS MODELOS ===")
models = ['Original', 'Hiperparámetros Optimizados', 'Pipeline Final', 'Feature Engineering', 'Combinación Final']
f1_scores = [f1_original, f1_optimized, f1_pipeline, f1_fe, f1_combined]
f1_by_class_all = [f1_by_class_original, f1_by_class_optimized, f1_by_class_pipeline, f1_by_class_fe, f1_by_class_combined]

# Crear DataFrame para resultados
results = pd.DataFrame({
    'Modelo': models,
    'F1-Score (Weighted)': f1_scores,
    'F1-Score (Clase 0)': [f[0] for f in f1_by_class_all],
    'F1-Score (Clase 1)': [f[1] for f in f1_by_class_all],
    'F1-Score (Clase 2)': [f[2] for f in f1_by_class_all],
})

# Calcular la desviación estándar de F1-scores entre clases (medida de equidad)
results['Desviación F1 entre clases'] = results[['F1-Score (Clase 0)', 'F1-Score (Clase 1)', 'F1-Score (Clase 2)']].std(axis=1)

print(results)

# Visualizar comparación de F1-scores
plt.figure(figsize=(14, 8))
bar_width = 0.15
index = np.arange(len(models))

plt.bar(index, results['F1-Score (Weighted)'], bar_width, label='F1 Weighted')
plt.bar(index + bar_width, results['F1-Score (Clase 0)'], bar_width, label='F1 Clase 0')
plt.bar(index + 2*bar_width, results['F1-Score (Clase 1)'], bar_width, label='F1 Clase 1')
plt.bar(index + 3*bar_width, results['F1-Score (Clase 2)'], bar_width, label='F1 Clase 2')

plt.xlabel('Modelo')
plt.ylabel('F1-Score')
plt.title('Comparación de F1-Scores por Modelo y Clase')
plt.xticks(index + 1.5*bar_width, models, rotation=15)
plt.legend()
plt.tight_layout()
plt.savefig('f1_comparison_decision_tree.png')
plt.close()

# Visualizar desviación estándar (equidad entre clases)
plt.figure(figsize=(12, 6))
plt.bar(models, results['Desviación F1 entre clases'], color='purple')
plt.xlabel('Modelo')
plt.ylabel('Desviación Estándar de F1 entre Clases')
plt.title('Equidad de Clasificación entre Clases (menor es mejor)')
plt.xticks(rotation=15)
plt.tight_layout()
plt.savefig('fairness_comparison_decision_tree.png')
plt.close()

# Identificar el mejor modelo
best_model_idx = results['F1-Score (Weighted)'].idxmax()
best_model_name = results.loc[best_model_idx, 'Modelo']
best_f1 = results.loc[best_model_idx, 'F1-Score (Weighted)']
best_fairness = results.loc[best_model_idx, 'Desviación F1 entre clases']

print(f"\n=== MEJOR MODELO: {best_model_name} ===")
print(f"F1-Score: {best_f1:.4f}")
print(f"Desviación entre clases: {best_fairness:.4f}")

# Comparar con el modelo original
original_f1 = results.loc[0, 'F1-Score (Weighted)']
original_fairness = results.loc[0, 'Desviación F1 entre clases']

print("\n=== COMPARACIÓN CON MODELO ORIGINAL ===")
print(f"Mejora en F1-Score: {(best_f1 - original_f1) / original_f1 * 100:.2f}%")
print(f"Mejora en equidad: {(original_fairness - best_fairness) / original_fairness * 100:.2f}%")

# Guardar resultados
results.to_csv('resultados_decision_tree.csv', index=False)
results.to_excel('resultados_decision_tree.xlsx', index=False)

print("\nResultados guardados en 'resultados_decision_tree.csv' y 'resultados_decision_tree.xlsx'")

=== DISTRIBUCIÓN ORIGINAL DE CLASES (TRAIN) ===
1    50
0    41
2    33
Name: count, dtype: int64
Proporción: 1    0.40
0    0.33
2    0.27
Name: proportion, dtype: float64

=== MODELO ORIGINAL (BASELINE) ===

=== RESULTADOS DEL MODELO: Original ===
F1-Score (weighted): 0.9632
F1-Score por clase: [0.97142857 0.95454545 0.96551724]

Matriz de Confusión:
[[17  1  0]
 [ 0 21  0]
 [ 0  1 14]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        18
           1       0.91      1.00      0.95        21
           2       1.00      0.93      0.97        15

    accuracy                           0.96        54
   macro avg       0.97      0.96      0.96        54
weighted avg       0.97      0.96      0.96        54


=== ESTRATEGIA 1: TRANSFORMACIÓN AVANZADA DE CARACTERÍSTICAS ===
F1-Score con PowerTransformer: 0.9632
F1-Score con QuantileTransformer: 0.9814
F1-Score con RobustScaler: 0.9632

Mejor transformador: